In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

In [54]:
%config InlineBackend.figure_format = "retina"
sns.set()

In [55]:
df = pd.read_csv("data_character_meaning.csv", index_col=0,
                 dtype={"success": "boolean"},
                 parse_dates=['ts_display', 'ts_reply'])

In [56]:
df

,user,domain,condition,item,item_character,item_meaning,success,teacher_md,learner_md,psy_md,...,is_eval,ts_display,ts_reply,n_session_done,pos_reply_0,pos_reply_1,pos_reply_2,pos_reply_3,pos_reply_4,pos_reply_5
0,goldfish@active.fi,active.fi,ThresholdCondition,1506,嬢,miss,True,leitner,NaN,grid,...,False,2020-09-30 17:51:37.666000+00:00,2020-09-30 17:51:41.042000+00:00,14,cape,cork,miss,stiff,strong,universal
1,goldfish@active.fi,active.fi,ThresholdCondition,190,姉,older sister,True,leitner,NaN,grid,...,False,2020-09-30 17:51:42.081000+00:00,2020-09-30 17:51:43.910000+00:00,14,age,aggression,conflict,measurement,miss,older sister
2,goldfish@active.fi,active.fi,ThresholdCondition,1000,婚,marriage,True,leitner,NaN,grid,...,False,2020-09-30 17:51:44.980000+00:00,2020-09-30 17:51:46.413000+00:00,14,comprehend,long,marriage,miss,older sister,reparation
3,goldfish@active.fi,active.fi,ThresholdCondition,1506,嬢,miss,True,leitner,NaN,grid,...,False,2020-09-30 17:51:47.476000+00:00,2020-09-30 17:51:50.206000+00:00,14,bright,fat,heat,marriage,miss,older sister
4,goldfish@active.fi,active.fi,ThresholdCondition,190,姉,older sister,True,leitner,NaN,grid,...,False,2020-09-30 17:51:51.245000+00:00,2020-09-30 17:51:53.477000+00:00,14,aim,marriage,miss,older sister,radiance,snow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85282,aster@active.fi,active.fi,ForwardCondition,1965,肖,resemblance,True,forward,exp_decay,grid,...,False,2020-10-06 12:33:07.610000+00:00,2020-10-06 12:33:11.043000+00:00,12,candy,distinction,grid,resemblance,season,ship
85283,aster@active.fi,active.fi,ForwardCondition,1429,斉,simultaneous,True,forward,exp_decay,grid,...,False,2020-10-06 12:33:17.376000+00:00,2020-10-06 12:33:19.775000+00:00,12,belt,die,fault,hermit,send,simultaneous
85284,aster@active.fi,active.fi,ForwardCondition,458,橋,bridge,True,forward,exp_decay,grid,...,False,2020-10-06 12:33:22.841000+00:00,2020-10-06 12:33:26.554000+00:00,12,bridge,condition,curtain,peak,previous,request
85285,aster@active.fi,active.fi,ForwardCondition,398,期,period of time,True,forward,exp_decay,grid,...,False,2020-10-06 12:33:32.375000+00:00,2020-10-06 12:33:34.619000+00:00,12,big,curtain,fault,grid,period of time,soldier


In [58]:
# Keep only users from the last experiment and that did it until the end (6 + 1  sessions for each teacher)
df.drop(df[(df.domain != "active.fi") | (df.n_session_done != 14)].index,
        inplace=True)

# Convert timestamps into seconds
beginning_history = pd.Timestamp("1970-01-01", tz="UTC")
df["timestamp"] = (df["ts_reply"] - beginning_history).dt.total_seconds().values

# Copy actual item ID in a new column
df["item_id"] = df.item
# Create new ids starting from zero
for i, i_id in enumerate(df.item_id.unique()):
    df.loc[df.item_id == i_id, 'item'] = i

# Total number of user
n_u = len(df.user.unique())

# Number of observations per user
n_o_by_u = np.zeros(shape=n_u, dtype=int)
for u, (user, user_df) in enumerate(df.groupby("user")):
    # Do not count first presentation
    n_o_by_u[u] = len(user_df) - len(user_df.item.unique()) 

# Total number of observation
n_obs = n_o_by_u.sum()

# Replies (1: success, 0: error)
y = np.zeros(shape=n_obs, dtype=int)
# Time elapsed since the last presentation of the same item (in seconds)
x = np.zeros(shape=n_obs, dtype=float)
# Number of repetition (number of presentation - 1)
r = np.zeros(shape=n_obs, dtype=int)
# Item ID
w = np.zeros(shape=n_obs, dtype=int)
# User ID
u = np.zeros(shape=n_obs, dtype=int)

# Fill the containers `y`, `x`, `r`, `w`, `u` 
idx = 0
for i_u, (user, user_df) in enumerate(df.groupby("user")):
    
    # Extract data from user `u`
    user_df = user_df.sort_values(by="timestamp")
    seen = user_df.item.unique()
    w_u = user_df.item.values
    ts_u = user_df.timestamp.values
    y_u = user_df.success.values
    
    # Initialize counts of repetition for each words at -1
    counts = {word: -1 for word in seen}
    # Initialize time of last presentation at None
    last_pres = {word: None for word in seen}
    
    # Number of observations for user `u` including first presentations
    n_obs_u_incl_first = len(user_df)
    
    # Number of repetitions for user `u`
    r_u = np.zeros(n_obs_u_incl_first)
    # Time elapsed since last repetition for user `u`
    x_u = np.zeros(n_obs_u_incl_first)  
    
    # Loop over each entry for user `u`:
    for i in range(n_obs_u_incl_first):
        
        # Get info for iteration `i`
        word = w_u[i]
        ts = ts_u[i]
        r_u[i] = counts[word]
        
        # Compute time elasped since last presentation
        if last_pres[word] is not None:
            x_u[i] = ts - last_pres[word]
        
        # Update count of repetition
        counts[word] += 1
        # Update last presentation
        last_pres[word] = ts
    
    # Keep only observations that are not the first presentation of an item
    to_keep = r_u >= 0
    y_u = y_u[to_keep]
    r_u = r_u[to_keep]
    w_u = w_u[to_keep]
    x_u = x_u[to_keep]
    
    # Number of observations for user `u` excluding first presentations
    n_obs_u = len(y_u)
    
    # Fill containers
    y[idx:idx + n_obs_u] = y_u
    x[idx:idx + n_obs_u] = x_u
    w[idx:idx + n_obs_u] = w_u
    r[idx:idx + n_obs_u] = r_u
    u[idx:idx + n_obs_u] = i_u
    
    # Update index
    idx += n_obs_u

n_w = len(np.unique(w))
n_o_max = n_o_by_u.max()
n_o_min = n_o_by_u.min()
print("number of user", n_u)
print("number of items", n_w)
print("total number of observations (excluding first presentation)", n_obs)
print("minimum number of observation for a single user", n_o_min)
print("maximum number of observation for a single user", n_o_max)

pd.DataFrame({
    'u': u, 'w': w,
    'x': x,
    'r': r,
    'y': y
})

number of user 53
number of items 1998
total number of observations (excluding first presentation) 70618
minimum number of observation for a single user 1285
maximum number of observation for a single user 1404


,u,w,x,r,y
0,0,1537,61.980,0,1
1,0,1967,116.744,0,1
2,0,1488,115.660,0,1
3,0,1198,138.515,0,1
4,0,563,154.035,0,1
...,...,...,...,...,...
70613,52,296,410.185,9,1
70614,52,606,418.810,11,1
70615,52,807,407.315,14,1
70616,52,538,374.817,10,1


In [59]:
# Average number of repetitions
r.mean()

6.4676711320060045

In [60]:
# Correct recall frequency
y.mean()

0.8626837350250645

In [63]:
# Minimum/Maximum time between two presentations (in seconds)
x.min(), x.max()

(1.7849998474121094, 476175.66499996185)